# GKS 2022 Data Visualization

## Importing libraries


In [14]:
#installation
##### This really depends for every computer, so please be caerful with the instalation
##!pip install konlpy
##!pip install pycountry-convert
##!pip install dataprep
###!pip install geopy

In [15]:
import pandas as pd
from konlpy.tag import Kkma, Hannanum, Komoran
from konlpy.utils import pprint
import pycountry_convert as pc
##import dataprep
from geopy.geocoders import Nominatim #function to get longitude and latitude data from country name

## Importing Data

In [3]:
df_gks = pd.read_csv("2022_GKS_Successful_Candidates.csv")

In [4]:
df_gks.head(10)

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status
0,CS01220218,Embassy,Albania,Agalliu Enrik,Master's,SEOKYEONG UNIVERSITY,Kyungpook National University,Electronic and Electrical Engineering,NaN
1,CS01220219,Embassy,Algeria,HADJ MOHAMMED Hanane,Doctoral,PAI CHAI UNIVERSITY,Seoul National University,Earth and Environmental Sciences,NaN
2,CS01220220,Embassy,Algeria,BOURAHLA Kamelia,Master's,SUNMOON UNIVERSITY,KOREA UNIVERSITY,Business Administration,NaN
3,CS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master's,SUNMOON UNIVERSITY,PUSAN NATIONAL UNIVERSITY,Dept. of Food and Resource Economics,NaN
4,CS01220222,Embassy,Argentina,Centurion Natali Egle,Master's,Keimyung University,Hankuk University of Foreign Studies,International Studies,NaN
5,CS01220223,Embassy,Argentina,Fernandez Carolina Belen,Master's,Konyang University,Sogang University,International Relations,NaN
6,CS01220224,Embassy,Armenia,Hayrapetyan Laura,Master's,PAI CHAI UNIVERSITY,Hankuk University of Foreign Studies,Korean Language and Literature,Conditional\r\nSuccessful Candidate
7,CS01220225,Embassy,Armenia,Martirosyan Milena,Master's,Busan University of Foreign\r\nStudies,Ewha Womans University,International Studies,NaN
8,CS01220226,Embassy,Australia,Gorey Michael Francis,Master's,exempt,Dongguk University,Film and Digital Media,NaN
9,CS01220227,Embassy,Austria,Wortha Samuel Roberto,Master's,exempt,KOREA UNIVERSITY,Media & Communication,Scholarship Deffered\r\n(6 months)


## Cleaning and processing Data

In [5]:
df_gks.describe()

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status
count,1193,1193,1193,1193,1193,1193,1193,1193,219
unique,1193,4,130,1193,5,32,59,459,3
top,CS01220218,Embassy,Indonesia,Agalliu Enrik,Master's,exempt,Hankuk University of Foreign Studies,International Studies,Conditional\r\nSuccessful Candidate
freq,1,506,99,1,929,236,56,44,149


In [6]:
df_gks['Korean Language Institution'] = df_gks['Korean Language Institution'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Korean Language Institution'] = df_gks['Korean Language Institution'].str.title()

df_gks['University'] = df_gks['University'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['University'] = df_gks['University'].str.title()

df_gks['Full Name (Family Given Middle)'] = df_gks['Full Name (Family Given Middle)'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Full Name (Family Given Middle)'] = df_gks['Full Name (Family Given Middle)'].str.title()

df_gks['Scholarship Status'] = df_gks['Scholarship Status'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Scholarship Status'] = df_gks['Scholarship Status'].str.capitalize()

df_gks['Department'] = df_gks['Department'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Department'] = df_gks['Department'].str.capitalize()

df_gks['Country'] = df_gks['Country'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Country'] = df_gks['Country'].str.title()

df_gks = df_gks.replace(to_replace="Master's", value="Master")


In [7]:
df_gks.head(10)

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status
0,CS01220218,Embassy,Albania,Agalliu Enrik,Master,Seokyeong University,Kyungpook National University,Electronic and electrical engineering,NaN
1,CS01220219,Embassy,Algeria,Hadj Mohammed Hanane,Doctoral,Pai Chai University,Seoul National University,Earth and environmental sciences,NaN
2,CS01220220,Embassy,Algeria,Bourahla Kamelia,Master,Sunmoon University,Korea University,Business administration,NaN
3,CS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master,Sunmoon University,Pusan National University,Dept. of food and resource economics,NaN
4,CS01220222,Embassy,Argentina,Centurion Natali Egle,Master,Keimyung University,Hankuk University Of Foreign Studies,International studies,NaN
5,CS01220223,Embassy,Argentina,Fernandez Carolina Belen,Master,Konyang University,Sogang University,International relations,NaN
6,CS01220224,Embassy,Armenia,Hayrapetyan Laura,Master,Pai Chai University,Hankuk University Of Foreign Studies,Korean language and literature,Conditionalsuccessful candidate
7,CS01220225,Embassy,Armenia,Martirosyan Milena,Master,Busan University Of Foreignstudies,Ewha Womans University,International studies,NaN
8,CS01220226,Embassy,Australia,Gorey Michael Francis,Master,Exempt,Dongguk University,Film and digital media,NaN
9,CS01220227,Embassy,Austria,Wortha Samuel Roberto,Master,Exempt,Korea University,Media & communication,Scholarship deffered(6 months)


In [8]:
df_gks = df_gks.replace(to_replace="Bosnia Andherzegovin", value="Bosnia and Herzegovina")
df_gks = df_gks.replace(to_replace="Burkinafaso", value="Burkina Faso")
df_gks = df_gks.replace(to_replace="Coted'Ivoire", value="Côte d'Ivoire")
df_gks = df_gks.replace(to_replace="Czechrepublic", value="Czechia")
df_gks = df_gks.replace(to_replace="Dominicanrepublic", value="Dominican Republic")
df_gks = df_gks.replace(to_replace="Dr Congo", value="Congo")
df_gks = df_gks.replace(to_replace="Idad And To", value="Trinidad and Tobago")
df_gks = df_gks.replace(to_replace="Turkmenista", value="Turkmenistan")
df_gks = df_gks.replace(to_replace="Uk", value="United Kingdom")
df_gks = df_gks.replace(to_replace="Usa", value="United States of America")
df_gks = df_gks.replace(to_replace="Minican Repu", value="Dominican Republic")

In [9]:
df_gks.head(10)

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status
0,CS01220218,Embassy,Albania,Agalliu Enrik,Master,Seokyeong University,Kyungpook National University,Electronic and electrical engineering,NaN
1,CS01220219,Embassy,Algeria,Hadj Mohammed Hanane,Doctoral,Pai Chai University,Seoul National University,Earth and environmental sciences,NaN
2,CS01220220,Embassy,Algeria,Bourahla Kamelia,Master,Sunmoon University,Korea University,Business administration,NaN
3,CS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master,Sunmoon University,Pusan National University,Dept. of food and resource economics,NaN
4,CS01220222,Embassy,Argentina,Centurion Natali Egle,Master,Keimyung University,Hankuk University Of Foreign Studies,International studies,NaN
5,CS01220223,Embassy,Argentina,Fernandez Carolina Belen,Master,Konyang University,Sogang University,International relations,NaN
6,CS01220224,Embassy,Armenia,Hayrapetyan Laura,Master,Pai Chai University,Hankuk University Of Foreign Studies,Korean language and literature,Conditionalsuccessful candidate
7,CS01220225,Embassy,Armenia,Martirosyan Milena,Master,Busan University Of Foreignstudies,Ewha Womans University,International studies,NaN
8,CS01220226,Embassy,Australia,Gorey Michael Francis,Master,Exempt,Dongguk University,Film and digital media,NaN
9,CS01220227,Embassy,Austria,Wortha Samuel Roberto,Master,Exempt,Korea University,Media & communication,Scholarship deffered(6 months)


In [10]:
rango = range(0, df_gks.shape[0])  ### 
country_code=list(rango)
continent_code =list(rango)

for i in range(df_gks.shape[0]):
    country_code[i] = pc.country_name_to_country_alpha2(df_gks.loc[i,'Country'], cn_name_format="default")
    #print(country_code[i])
    
country_code = list(map(lambda item: item.replace("TL","TH"), country_code))
    
for i in range(df_gks.shape[0]):    
    continent_code[i] = pc.country_alpha2_to_continent_code(country_code[i])
    ##print(continent_code[i])    

In [11]:
df_gks['alpha2_country'] = country_code
df_gks['alpha2_cotinent'] = continent_code

In [17]:
df_gks.head(10)

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status,alpha2_country,alpha2_cotinent
0,CS01220218,Embassy,Albania,Agalliu Enrik,Master,Seokyeong University,Kyungpook National University,Electronic and electrical engineering,NaN,AL,EU
1,CS01220219,Embassy,Algeria,Hadj Mohammed Hanane,Doctoral,Pai Chai University,Seoul National University,Earth and environmental sciences,NaN,DZ,AF
2,CS01220220,Embassy,Algeria,Bourahla Kamelia,Master,Sunmoon University,Korea University,Business administration,NaN,DZ,AF
3,CS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master,Sunmoon University,Pusan National University,Dept. of food and resource economics,NaN,AO,AF
4,CS01220222,Embassy,Argentina,Centurion Natali Egle,Master,Keimyung University,Hankuk University Of Foreign Studies,International studies,NaN,AR,SA
5,CS01220223,Embassy,Argentina,Fernandez Carolina Belen,Master,Konyang University,Sogang University,International relations,NaN,AR,SA
6,CS01220224,Embassy,Armenia,Hayrapetyan Laura,Master,Pai Chai University,Hankuk University Of Foreign Studies,Korean language and literature,Conditionalsuccessful candidate,AM,AS
7,CS01220225,Embassy,Armenia,Martirosyan Milena,Master,Busan University Of Foreignstudies,Ewha Womans University,International studies,NaN,AM,AS
8,CS01220226,Embassy,Australia,Gorey Michael Francis,Master,Exempt,Dongguk University,Film and digital media,NaN,AU,OC
9,CS01220227,Embassy,Austria,Wortha Samuel Roberto,Master,Exempt,Korea University,Media & communication,Scholarship deffered(6 months),AT,EU


In [32]:
geolocator = Nominatim(user_agent="pizza")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_gks['location'] = df_gks['alpha2_country'].apply(geocode)
df_gks['point'] = df_gks['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('GR',), **{}).
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\ProgramData\Anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read

RateLimiter swallowed an error after 2 retries. Called with (*('GR',), **{}).
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\ProgramData\Anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nb

RateLimiter caught an error, retrying (1/2 tries). Called with (*('GR',), **{}).
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\ProgramData\Anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\ProgramData\Anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read

KeyboardInterrupt: 

In [34]:
geolocator = Nominatim(user_agent="pizza")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_gks['location'] = df_gks['Country'].apply(geocode)
df_gks['point'] = df_gks['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [36]:
df_gks.head(50)

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status,alpha2_country,alpha2_cotinent,location,point
0,CS01220218,Embassy,Albania,Agalliu Enrik,Master,Seokyeong University,Kyungpook National University,Electronic and electrical engineering,NaN,AL,EU,"(Shqipëria, (41.000028, 19.9999619))","(41.000028, 19.9999619, 0.0)"
1,CS01220219,Embassy,Algeria,Hadj Mohammed Hanane,Doctoral,Pai Chai University,Seoul National University,Earth and environmental sciences,NaN,DZ,AF,"(Algérie / ⵍⵣⵣⴰⵢⴻⵔ / الجزائر, (28.0000272, 2.9...","(28.0000272, 2.9999825, 0.0)"
2,CS01220220,Embassy,Algeria,Bourahla Kamelia,Master,Sunmoon University,Korea University,Business administration,NaN,DZ,AF,"(Algérie / ⵍⵣⵣⴰⵢⴻⵔ / الجزائر, (28.0000272, 2.9...","(28.0000272, 2.9999825, 0.0)"
3,CS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master,Sunmoon University,Pusan National University,Dept. of food and resource economics,NaN,AO,AF,"(Angola, (-11.8775768, 17.5691241))","(-11.8775768, 17.5691241, 0.0)"
4,CS01220222,Embassy,Argentina,Centurion Natali Egle,Master,Keimyung University,Hankuk University Of Foreign Studies,International studies,NaN,AR,SA,"(Argentina, (-34.9964963, -64.9672817))","(-34.9964963, -64.9672817, 0.0)"
5,CS01220223,Embassy,Argentina,Fernandez Carolina Belen,Master,Konyang University,Sogang University,International relations,NaN,AR,SA,"(Argentina, (-34.9964963, -64.9672817))","(-34.9964963, -64.9672817, 0.0)"
6,CS01220224,Embassy,Armenia,Hayrapetyan Laura,Master,Pai Chai University,Hankuk University Of Foreign Studies,Korean language and literature,Conditionalsuccessful candidate,AM,AS,"(Հայաստան, (40.7696272, 44.6736646))","(40.7696272, 44.6736646, 0.0)"
7,CS01220225,Embassy,Armenia,Martirosyan Milena,Master,Busan University Of Foreignstudies,Ewha Womans University,International studies,NaN,AM,AS,"(Հայաստան, (40.7696272, 44.6736646))","(40.7696272, 44.6736646, 0.0)"
8,CS01220226,Embassy,Australia,Gorey Michael Francis,Master,Exempt,Dongguk University,Film and digital media,NaN,AU,OC,"(Australia, (-24.7761086, 134.755))","(-24.7761086, 134.755, 0.0)"
9,CS01220227,Embassy,Austria,Wortha Samuel Roberto,Master,Exempt,Korea University,Media & communication,Scholarship deffered(6 months),AT,EU,"(Österreich, (47.59397, 14.12456))","(47.59397, 14.12456, 0.0)"


In [41]:
#installation
##pip install geopy
#function to get longitude and latitude data from country name
from geopy.geocoders import Nominatim
##geolocator = Nominatim()
def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan

In [44]:
df_gks.describe()

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status,alpha2_country,alpha2_cotinent,location,point
count,1193,1193,1193,1193,1193,1193,1193,1193,219,1193,1193,1193,1193
unique,1193,4,129,1193,5,32,59,437,3,128,6,129,129
top,CS01220218,Embassy,Indonesia,Agalliu Enrik,Master,Exempt,Hankuk University Of Foreign Studies,Business administration,Conditionalsuccessful candidate,ID,AS,"(Indonesia, (-2.4833826, 117.8902853))","(-2.4833826, 117.8902853, 0.0)"
freq,1,506,99,1,929,236,56,44,149,99,833,99,99


In [37]:
df_gks.to_csv(index=False)

'CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status,alpha2_country,alpha2_cotinent,location,point\r\nCS01220218,Embassy,Albania,Agalliu Enrik,Master,Seokyeong University,Kyungpook National University,Electronic and electrical engineering,,AL,EU,Shqipëria,"(41.000028, 19.9999619, 0.0)"\r\nCS01220219,Embassy,Algeria,Hadj Mohammed Hanane,Doctoral,Pai Chai University,Seoul National University,Earth and environmental sciences,,DZ,AF,Algérie / ⵍⵣⵣⴰⵢⴻⵔ / الجزائر,"(28.0000272, 2.9999825, 0.0)"\r\nCS01220220,Embassy,Algeria,Bourahla Kamelia,Master,Sunmoon University,Korea University,Business administration,,DZ,AF,Algérie / ⵍⵣⵣⴰⵢⴻⵔ / الجزائر,"(28.0000272, 2.9999825, 0.0)"\r\nCS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master,Sunmoon University,Pusan National University,Dept. of food and resource economics,,AO,AF,Angola,"(-11.8775768, 17.5691241, 0.0)"\r\nCS01220222,Embassy,Argentina,Centurion 

In [39]:
from pathlib import Path  
filepath = Path('D:/Coreano/jeong/koreanLanguage/salida.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_gks.to_csv(filepath)  

In [29]:
##import pandas as pd
df = pd.DataFrame({'name': ['paris', 'berlin', 'london']})

##from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="pizza")

##from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['location'] = df['name'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [26]:
df.head(10)

,name,location,point
0,paris,"(Paris, Île-de-France, France métropolitaine, ...","(48.8588897, 2.3200410217200766, 0.0)"
1,berlin,"(Berlin, Deutschland, (52.5170365, 13.3888599))","(52.5170365, 13.3888599, 0.0)"
2,london,"(London, Greater London, England, United Kingd...","(51.5073219, -0.1276474, 0.0)"
